In [1]:
import os
os.chdir('../')

In [2]:
%matplotlib inline
#%matplotlib notebook

%load_ext autoreload
%autoreload 2

In [3]:
from copy import deepcopy
from decimal import Decimal
from typing import List, Tuple
from warnings import warn
import resource

from cycler import cycler
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.decomposition import PCA
import scipy.stats as stats
import torch
from torch import nn
import torch.nn.functional as F
import seml
from ogb.nodeproppred import PygNodePropPredDataset

import tqdm
tqdm.tqdm.pandas()

In [4]:
from notebooks import mpl_latex

/nfs/staff-ssd/geisler/anaconda3/envs/rgnn_at_scale_refactoring/lib/python3.8/_collections_abc.py:832: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


In [5]:
mpl_latex.enable_production_mode()

In [6]:
from rgnn_at_scale.data import prep_graph, split
from rgnn_at_scale.attacks import create_attack, SPARSE_ATTACKS
from rgnn_at_scale.helper.io import Storage
from rgnn_at_scale.models import DenseGCN, GCN
from rgnn_at_scale.train import train
from rgnn_at_scale.helper.utils import accuracy

In [36]:
dataset_map = {
    'cora_ml': r'\textbf{Cora ML}',#'~\citep{Bojchevski2018}}', # '\rotatebox{90}{Cora ML~\citep{Bojchevski2018}}',
    'citeseer': r'\textbf{Citeseer}',#~\citep{McCallum2000}}',
    'pubmed': r'\textbf{PubMed}',#~\citep{Sen2008}}',
    'ogbn-arxiv': r'\textbf{arXiv}',#~\citep{Hu2020}}',
    'ogbn-products': r'\textbf{Products}',#~\citep{Hu2020}}',
    'ogbn-papers100M': r'\textbf{Papers 100M}',#~\citep{Hu2020}}'
}
dataset_order = [dataset_map[k] for k in dataset_map.keys()]

In [8]:
datasets = ['cora_ml']#, 'pubmed', 'ogbn-arxiv']
search_space_sizes = [1_000_000, 2_500_000, 10_000_000]
binary_attr = False
attack = 'PRBCD'
seed = 0
device = 0
victim_param_list = [
    {
        'n_filters': 64,
        'dropout': 0.5
    },
    {
        'n_filters': 64,
        'dropout': 0.5
    },
    {
        'n_filters': 64,
        'dropout': 0.5
    },
    {
        'n_filters': [256, 256],
        'dropout': 0.5,
        'with_batchnorm': True,
    }
]
train_params = {
    'lr': 1e-2,
    'weight_decay': 0,
    'patience': 300,
    'max_epochs': 3000
}
attack_params = {
    'keep_heuristic': 'WeightOnly',
    'loss_type': 'tanhMargin',
    'fine_tune_epochs': 0,
    'with_early_stropping': False
}
epsilon = 0.1
display_steps = 10

make_undirected = True
loss_type = 'tanhMargin'

torch.manual_seed(seed)
np.random.seed(seed)

reruns = 5

In [9]:
results = []

for dataset, search_space_size, victim_params in zip(datasets, search_space_sizes, victim_param_list):
    for rerun in range(reruns):
        tmp_result = dict(dataset=dataset, search_space_size=search_space_size, rerun=rerun)
        print(dataset, search_space_size, victim_params, make_undirected)

        graph = prep_graph(dataset, device='cpu', binary_attr=binary_attr, make_undirected=make_undirected,
                           return_original_split=dataset.startswith('ogbn'))

        tmp_result['main_memory_before'] = 1024 * resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
        tmp_result['cuda_reserved_before'] = torch.cuda.max_memory_reserved() / 1e9
        tmp_result['cuda_allocated_before'] = torch.cuda.max_memory_allocated() / 1e9

        attr, adj, labels = graph[:3]
        if len(graph) == 3:
            idx_train, idx_val, idx_test = split(labels.cpu().numpy())
        else:
            idx_train, idx_val, idx_test = graph[3]['train'], graph[3]['valid'], graph[3]['test']

        n_features = attr.shape[1]
        n_classes = int(labels.max() + 1)

        victim_params_current = dict(victim_params)
        if make_undirected == False:
            victim_params['row_norm'] = True
            victim_params_current['n_filters'] = []

        gcn = GCN(n_classes=n_classes, n_features=n_features, **victim_params_current).to(device)

        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)

        start.record()
        traces = train(model=gcn, attr=attr.to(device), adj=adj.to(device), labels=labels.to(device),
                       idx_train=idx_train, idx_val=idx_val, display_step=display_steps, **train_params)
        end.record()
        torch.cuda.synchronize()

        tmp_result['time_train'] = start.elapsed_time(end) / 1000
        tmp_result['time_epoch_train'] = tmp_result['time_train'] / len(traces[0])

        tmp_result['main_memory_train'] = 1024 * resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
        tmp_result['cuda_reserved_train'] = torch.cuda.max_memory_reserved() / 1e9
        tmp_result['cuda_allocated_train'] = torch.cuda.max_memory_allocated() / 1e9

        gcn.eval()

        with torch.no_grad():
            clean_logits = gcn(attr.to(device), adj.to(device))
        accuracy(clean_logits, labels.to(device), idx_test)
        print(f'Clean accuracy: {accuracy(clean_logits, labels.to(device), idx_test)}')

        print(dataset, make_undirected, epsilon, loss_type)

        temp_attack_params = dict(attack_params)
        temp_attack_params['loss_type'] = loss_type
        temp_attack_params['epsilon'] = epsilon
        temp_attack_params['search_space_size'] = search_space_size

        adversary = create_attack(attack, adj=adj, attr=attr, binary_attr=False, labels=labels, 
                                  make_undirected=make_undirected, model=gcn, idx_attack=idx_test, 
                                  device=device, data_device=device, **temp_attack_params)

        m = adj.nnz()
        torch.manual_seed(seed)
        np.random.seed(seed)

        n_perturbations = int(round(epsilon * m))

        start.record()
        adversary.attack(n_perturbations)
        end.record()
        torch.cuda.synchronize()

        tmp_result['time_attack'] = start.elapsed_time(end) / 1000
        tmp_result['time_epoch_attack'] = tmp_result['time_attack'] / len(adversary.attack_statistics['loss'])

        tmp_result['main_memory_attack'] = 1024 * resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
        tmp_result['cuda_reserved_attack'] = torch.cuda.max_memory_reserved() / 1e9
        tmp_result['cuda_allocated_attack'] = torch.cuda.max_memory_allocated() / 1e9

        with torch.no_grad():
            attacked_logits = gcn(attr.to(device), adversary.adj_adversary.to(device))
        print(f'Perturbed accuracy: {accuracy(attacked_logits, labels.to(device), idx_test)}')

        results.append(tmp_result)

cora_ml 1000000 {'n_filters': 64, 'dropout': 0.5} True


Training...:   0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 1/400 [00:00<00:51,  7.82it/s]

Clean accuracy: 0.790513813495636
cora_ml True 0.1 tanhMargin

Before the attack - Loss: -0.5749706625938416 Accuracy: 79.051 %


Epoch: 0 Loss: -0.5749711394309998 Accuracy: 68.419 %



  6%|▌         | 22/400 [00:02<00:43,  8.75it/s]

...


Epoch: 360 Loss: 0.6442639231681824 Accuracy: 17.332 %



 95%|█████████▌| 381/400 [09:48<00:29,  1.57s/it]


Epoch: 380 Loss: 0.6498628854751587 Accuracy: 17.055 %



100%|██████████| 400/400 [10:18<00:00,  1.55s/it]


Perturbed accuracy: 0.18268419802188873


In [85]:
dataset_c = r'\textbf{Dataset}'
search_space_size_c = r'\textbf{Block size $b$}'
cuda_allocated_attack_c = r'\textbf{Max GPU memory \textbackslash GB}'
time_epoch_attack_c = r'\textbf{Duration of epoch \textbackslash s}'

df_results = pd.DataFrame(results).rename(columns={'dataset': dataset_c, 
                                                   'cuda_allocated_attack': cuda_allocated_attack_c, 
                                                   'time_epoch_attack': time_epoch_attack_c,
                                                   'search_space_size': search_space_size_c})

df_results[dataset_c] = df_results[dataset_c].apply(lambda dataset: dataset_map[dataset]).astype("category")
df_results[dataset_c].cat.set_categories(dataset_order, inplace=True)

df_results = df_results.groupby(dataset_c).agg({
    search_space_size_c: 'max', cuda_allocated_attack_c: 'max', time_epoch_attack_c: 'mean'
})

df_results[search_space_size_c] = df_results[search_space_size_c].apply(lambda b: f'{b:,}'[:-2])
df_results = df_results.round(3).reset_index().dropna()

df_results

,\textbf{Dataset},\textbf{Block size $b$},\textbf{Max GPU memory \ GB},\textbf{Duration of epoch \ s}
0,\textbf{Cora ML},"1,000,000",0.340,0.117
2,\textbf{PubMed},"2,500,000",0.889,0.321
3,\textbf{arXiv},"10,000,000",4.286,1.552


In [86]:
print(df_results
    .sort_index()
    .to_latex(
        float_format='%.2f',
        caption=r'Time cost and memory cost of our \emph{global} PR-BCD attack on a Vanilla GCN.', 
        label='tab:appendix_globaltimememory',
        escape=False,
        index=False
    )
)

\begin{table}
\centering
\caption{Time cost and memory cost of our \emph{global} PR-BCD attack on a Vanilla GCN.}
\label{tab:appendix_globaltimememory}
\begin{tabular}{llrr}
\toprule
\textbf{Dataset} & \textbf{Block size $b$} &  \textbf{Max GPU memory \ GB} &  \textbf{Duration of epoch \ s} \\
\midrule
\textbf{Cora ML} &               1,000,000 &                          0.34 &                            0.12 \\
 \textbf{PubMed} &               2,500,000 &                          0.89 &                            0.32 \\
  \textbf{arXiv} &              10,000,000 &                          4.29 &                            1.55 \\
\bottomrule
\end{tabular}
\end{table}



# Local attacks

In [87]:
labels_to_plot = [
    'Soft Median RPPRGo (T=1.0)',
    'Soft Median PPRGo (T=1.0)',
    'Soft Median PPRGo (T=100.0)',
    'Vanilla PPRGo',
    'Vanilla PPRGo LowAcc'
]
label_order = ['Soft Median PPRGo', 'Vanilla PPRGo']
def transform_label(label: str):
    return (
        label.replace('\n', ' ')
        .replace('RPPRGo', 'PPRGo')
        .replace(' (rank=50)', '')
        .replace(' (T=0.2)','')
        .replace(' (T=0.5)','')
        .replace(' (T=1.0)','')
        .replace(' (T=5.0)','')
        .replace(' (T=100.0)','')
        .replace(' LowAcc','')
    )

pd.read_csv('cache/memory_runtime_stats.csv')['label'].unique()

array(['Soft Median RPPRGo (T=1.0)', 'Vanilla PPRGo',
       'Soft Median PPRGo (T=100.0)', 'Soft Median PPRGo (T=20.0) LowAcc',
       'Soft Median PPRGo (T=200.0)', 'Soft Median PPRGo (T=50.0)',
       'Soft Median PPRGo (T=500.0)', 'Vanilla PPRGo LowAcc',
       'Soft Median PPRGo (T=1.0)', 'Vanilla GCN'], dtype=object)

In [88]:
architecture_c = r'\textbf{Victim}'

df_local = pd.read_csv('cache/memory_runtime_stats.csv')

df_local = df_local.rename(columns={'config.dataset': dataset_c, 
                                    'stats.pytorch.gpu_max_memory_bytes_mean': cuda_allocated_attack_c, 
                                    'node_eps_epoch_real_time_mean': time_epoch_attack_c,
                                    'config.attack_params.search_space_size_mean': search_space_size_c,
                                    'label': architecture_c})

df_local[dataset_c] = df_local[dataset_c].apply(lambda dataset: dataset_map[dataset]).astype("category")
df_local[dataset_c].cat.set_categories(dataset_order, inplace=True)

df_local = df_local[df_local[architecture_c].isin(labels_to_plot)]
df_local[architecture_c] = df_local[architecture_c].apply(lambda label: transform_label(label)).astype("category")
df_local[architecture_c].cat.set_categories(label_order, inplace=True)

df_local[search_space_size_c] = df_local[search_space_size_c].apply(lambda b: f'{b:,}'[:-2])
df_local[cuda_allocated_attack_c] = df_local[cuda_allocated_attack_c] / 1e9
df_local = df_local.round(3).reset_index().dropna()

df_local = df_local[df_local['config.attack'] == 'LocalBatchedPRBCD'].set_index([dataset_c, architecture_c])
df_local = df_local[[search_space_size_c, cuda_allocated_attack_c, time_epoch_attack_c]]
#df_local = df_local.reset_index()

df_local

\textbf{Block size $b$}  \
\textbf{Dataset}     \textbf{Victim}                             
\textbf{Citeseer}    Soft Median PPRGo                  10,000   
                     Vanilla PPRGo                      10,000   
\textbf{Cora ML}     Soft Median PPRGo                  10,000   
                     Vanilla PPRGo                      10,000   
\textbf{Papers 100M} Soft Median PPRGo                   2,500   
                     Vanilla PPRGo                       2,500   
\textbf{Products}    Soft Median PPRGo                  20,000   
                     Vanilla PPRGo                      20,000   

                                        \textbf{Max GPU memory \ GB}  \
\textbf{Dataset}     \textbf{Victim}                                   
\textbf{Citeseer}    Soft Median PPRGo                         0.078   
                     Vanilla PPRGo                             0.136   
\textbf{Cora ML}     Soft Median PPRGo                         0.078   
                     Vanilla PPRGo                             0.125   
\textbf{Papers 100M} Soft Median PPRGo                         2.847   
                     Vanilla PPRGo                             2.815   
\textbf{Products}    Soft Median PPRGo                         5.581   
                     Vanilla PPRGo                             6.067   

                                        \textbf{Duration of epoch \ s}  
\textbf{Dataset}     \textbf{Victim}                                    
\textbf{Citeseer}    Soft Median PPRGo                           0.385  
                     Vanilla PPRGo                               0.470  
\textbf{Cora ML}     Soft Median PPRGo                           0.489  
                     Vanilla PPRGo                               0.523  
\textbf{Papers 100M} Soft Median PPRGo                          14.664  
                     Vanilla PPRGo                              14.820  
\textbf{Products}    Soft Median PPRGo                           4.288  
                     Vanilla PPRGo                               3.258

In [89]:
print(df_local
    .sort_index()
    .to_latex(
        float_format='%.2f',
        caption=r'Time cost and memory cost of our \emph{local} PR-BCD attack on PPRGo.', 
        label='tab:appendix_localtimememory',
        escape=False
    )
)

\begin{table}
\centering
\caption{Time cost and memory cost of our \emph{local} PR-BCD attack on PPRGo.}
\label{tab:appendix_localtimememory}
\begin{tabular}{lllrr}
\toprule
                     &               & \textbf{Block size $b$} &  \textbf{Max GPU memory \ GB} &  \textbf{Duration of epoch \ s} \\
\textbf{Dataset} & \textbf{Victim} &                         &                               &                                 \\
\midrule
\textbf{Cora ML} & Soft Median PPRGo &                  10,000 &                          0.08 &                            0.49 \\
                     & Vanilla PPRGo &                  10,000 &                          0.12 &                            0.52 \\
\textbf{Citeseer} & Soft Median PPRGo &                  10,000 &                          0.08 &                            0.39 \\
                     & Vanilla PPRGo &                  10,000 &                          0.14 &                            0.47 \\
\textbf{Products} & Soft M